In [ ]:
%matplotlib widget
from sympy import *
init_printing(use_latex=True)

# 14 - Series Expansion

## 14.1 - Basic Usage

In [ ]:
x = symbols("x")
s = series(sin(x))
s

In [ ]:
s = log(x).series(x, 2, 4)
s

### 14.1.1 - The Order class and the removeO() method

In [ ]:
o = s.args[4]
display(o)
type(o)

In [ ]:
o.args

In [ ]:
Order(x + x**2, (x, 0))

In [ ]:
O(x + x**2, (x, oo))

In [ ]:
expr1 = x + x**3 + x**5 + x**7
expr2 = expr1 + Order(x**4)
display(expr1, expr2)

In [ ]:
s1 = cos(x).series(x, pi, 6)
s2 = s1.removeO()
display(s1, s2)

### 14.1.2 - Series expansion of multivariate expressions

In [ ]:
y = symbols("y")
expr = sin(x) * cos(y)
expr.series(x, 0).series(y, 0)

In [ ]:
expr.series(x, 0).removeO().series(y, 0).removeO()

### 14.1.3 - Series expansion of Undefined Functions

In [ ]:
f = Function("f")
s = f(x).series(x, 0, 3)
s

In [ ]:
t = s.args[0].args[1]
display(t)
print(type(t))

In [ ]:
s.args

In [ ]:
x0 = symbols("x_0")
s = f(x).series(x, x0, 3)
s

In [ ]:
s.subs(f, sin).doit()

## Fourier Expansion

In [ ]:
x, L = symbols("x, L")
expr = x / (2 * L)
fs = fourier_series(expr, (x, 0, 2 * L))
display(expr, fs)
print(type(fs))

In [ ]:
n = 3
display(fs.truncate(n))

In [ ]:
from spb import plot
plot((frac(x / 2), "sawtooth"),
      (fs.truncate(2).subs(L, 1), "n=2"),
      (fs.truncate(4).subs(L, 1), "n=4"),
      (fs.truncate(10).subs(L, 1), "n=10"),
      (x, -0.5, 2.5))

## 14.2 - Example - Linearization

In [ ]:
t = symbols("t")
phi, psi, theta = (s(t) for s in symbols("phi, psi, theta", cls=Function))
expr = -cos(phi) * sin(psi) + sin(phi) * sin(theta) * cos(psi)
expr

In [ ]:
expr.series(phi, 0)

In [ ]:
def linearize(expr, order=1, tup=None, n=3, apply_lin=True):
    if n < order:
        n = order
    if not tup:
        from sympy.core.function import AppliedUndef
        tup = []
        funcs = list(expr.find(AppliedUndef))
        for f in funcs:
            tup.append((f, 0))

    subs_dict = dict()
    for t in tup:
        f, f0 = t
        s = Dummy(f.func.name)
        expr = expr.subs(f, s)
        subs_dict[s] = f
        expr = expr.series(s, f0, n).removeO()

    if apply_lin:
        expr = expr.expand()
        get_degree = lambda expr, symbols: sum([degree(expr, gen=s) for s in symbols])
        args = [a for a in expr.args if get_degree(a, list(subs_dict.keys())) <= order]
        expr = expr.func(*args)
    expr = expr.subs(subs_dict)
    return expr

In [ ]:
linearize(expr)

In [ ]:
linearize(expr, apply_lin=False)

In [ ]:
linearize(expr, 2)